***

### **Project Report Disposition**

# Project Report: Physics-Informed Neural Operators for Surrogate Modeling of Reservoir Simulation

**Author:** Bjørn Egil Ludvigsen
**Course:** Advanced Machine Learning for the Physical Sciences

---

### **Abstract**
*(Target: ~0.5 pages | Grading Points: 5)*
* **Motivation & Problem:** The critical need for computationally efficient yet accurate models in reservoir management and the prohibitive cost of traditional numerical simulators (e.g., Eclipse).
* **Methodology:** Introduction of Physics-Informed Neural Operators (PINO) as a data-driven, physics-constrained surrogate modeling technique.
* **Implementation:** Brief overview of the project's core tasks: (1) creating a data processing pipeline to handle an ensemble of Eclipse simulation results, and (2) training a PINO model to map static reservoir properties to dynamic pressure/saturation responses.
* **Key Results:** A high-level summary of the PINO model's performance, highlighting its accuracy (e.g., low relative error) and the significant computational speed-up achieved compared to the numerical simulator.
* **Conclusion:** Statement on the potential of PINO to accelerate key reservoir engineering workflows like uncertainty quantification and optimization.

---

### **1. Introduction**
*(Target: ~1.5-2 pages | Grading Points: 10)*
* **1.1 The Role of Reservoir Simulation:** Context on its importance in field development, forecasting, and decision-making, governed by multiphase flow physics and solved by numerical simulators like Eclipse.
* **1.2 The Computational Bottleneck in Reservoir Management:** Discussion of the high computational cost and its impact on workflows like history matching, uncertainty quantification (UQ), and optimization.
* **1.3 Surrogate Modeling as a Solution:** Overview of surrogate models, from traditional methods to modern deep learning approaches, as a means to accelerate these workflows.
* **1.4 Physics-Informed Neural Operators (PINO):** Introduction to PINO, building on Fourier Neural Operators (FNO), which integrates governing PDEs into the training process to improve data efficiency, generalization, and physical plausibility.
* **1.5 Project Objectives:** A clear list of goals: (1) Build a data pipeline for Eclipse `.UNRST` files. (2) Implement and train a PINO model. (3) Evaluate the model's accuracy and performance. (4) Analyze the results and the method's applicability.

---

### **2. Formalism and Methods**
*(Target: ~3-4 pages | Grading Points: 20)*
* **2.1 Governing Equations of Two-Phase Flow:** Presentation of the relevant PDEs (mass conservation + Darcy's Law) and key variables (pressure, saturation, permeability).
* **2.2 Numerical Simulation with Eclipse:** Explanation of the discretization methods and the Eclipse file structure, justifying the use of restart files (`.UNRST`).
* **2.3 Data Preparation and Preprocessing Pipeline:** Details on the ensemble data, the Python-based parsing of binary files, feature engineering (input: permeability, output: pressure/saturation series), and data normalization.
* **2.4 Physics-Informed Neural Operator (PINO) Architecture:** A walkthrough of the FNO architecture and a detailed explanation of the PINO loss function, combining a data-mismatch term (`L_data`) and a physics-residual term (`L_phys`).

---

### **3. Code, Implementation, and Testing**
*(Target: ~3-4 pages | Grading Points: 20)*
* **3.1 Development Environment:** List of key software (Python) and libraries (PyTorch).
* **3.2 Implementation of Data Pipeline:** Commented Python code snippets for reading `.UNRST` files and creating PyTorch tensors.
* **3.3 PINO Model Implementation:** Code defining the PINO model, the physics-loss function using `torch.autograd`, and the training loop algorithm.
* **3.4 Training and Validation Protocol:** Description of the train/validation/test split and key hyperparameters.
* **3.5 Testing and Benchmarking:** Procedure for testing on unseen data and definition of evaluation metrics (Relative L2 error for accuracy, wall-clock time for performance).

---

### **4. Analysis and Discussion of Results**
*(Target: ~3-4 pages | Grading Points: 20)*
* **4.1 Model Accuracy Evaluation:** Visual (contour plots) and quantitative (error metrics over time) comparison of PINO predictions vs. Eclipse ground truth.
* **4.2 Impact of the Physics-Informed Loss:** A comparative study between a baseline FNO and the full PINO to demonstrate the benefits of the physics loss.
* **4.3 Computational Performance Benchmark:** A table summarizing and comparing the execution times (PINO training, PINO inference, Eclipse simulation) and the resulting speed-up factor.
* **4.4 Interpretation and Understanding:** Discussion of *why* the results are as they are, analyzing the model's ability to capture physical phenomena and any observed discrepancies.

---

### **5. Conclusions and Future Directions**
*(Target: ~1-1.5 pages | Grading Points: 10)*
* **5.1 Summary of Findings:** A recap of project outcomes and lessons learned about data preparation and the PINO method.
* **5.2 Limitations of the Study:** Honest assessment of the project's scope (e.g., 2D simplification, limited ensemble size).
* **5.3 Avenues for Future Work:** Suggestions for extensions, such as scaling to 3D, incorporating more complex physics, or applying the surrogate to a practical optimization workflow.

---

### **6. References**
*(Target: ~0.5-1 page | Grading Points: 5)*
* A formatted list of all cited scientific papers, books, and software documentation.

---

### **7. Appendix (Optional)**
* Could include more extensive code blocks, additional result plots, or detailed derivations if necessary.

***
***



### **Chapter 1: Introduction**

# 1. Introduction

## 1.1 The Role of Reservoir Simulation

Reservoir simulation stands as a cornerstone of modern petroleum engineering, providing the essential tools to model and predict the complex behavior of fluids within subterranean porous media. These simulations are fundamental to making informed, high-stakes decisions throughout the lifecycle of an oil or gas field. During the exploration and appraisal phases, simulation helps estimate reserves and assess the economic viability of a discovery. For field development planning, simulators are used to optimize key factors such as the number, location, and type of wells, as well as to design efficient production strategies and recovery mechanisms [Cite industry standard textbook, e.g., Ertekin, Abou-Kassem & King, 2001].

The underlying physical principles governing this subsurface flow are captured by a system of coupled, non-linear partial differential equations (PDEs). These equations describe the conservation of mass for each fluid phase (e.g., oil, water, gas) and relate fluid flux to pressure gradients via Darcy's Law. Numerical simulators, such as the industry-standard Eclipse software, solve these complex PDEs by discretizing the reservoir domain into a grid of cells and stepping forward in time. The output of these simulations provides engineers with dynamic, four-dimensional (3D space + time) maps of properties like pressure, saturation, and fluid composition, which are invaluable for managing reservoir performance and maximizing hydrocarbon recovery.

## 1.2 The Computational Bottleneck in Reservoir Management

Despite their critical importance, high-fidelity numerical simulations are notoriously computationally expensive. A single simulation run for a large, geologically complex model can take hours, days, or even weeks to complete, even with access to high-performance computing resources. This computational cost creates a significant bottleneck for many essential reservoir engineering workflows that inherently require a large number of simulation runs.

Key examples include:
* **History Matching:** The process of calibrating a reservoir model by adjusting its parameters (e.g., permeability and porosity) to match historical production data. This is an inverse problem that often requires thousands of simulation runs to find an acceptable match.
* **Uncertainty Quantification (UQ):** Assessing the impact of static geological and dynamic flow uncertainty on production forecasts. This involves running simulations on an ensemble of hundreds or thousands of plausible reservoir models to generate a probabilistic forecast.
* **Optimization:** Finding the optimal well placement or operational strategy to maximize an objective function, such as Net Present Value (NPV). This also requires an iterative process of running numerous simulations.

The prohibitive time cost of simulation often forces engineers to work with a limited number of scenarios, potentially leading to sub-optimal decisions and an incomplete understanding of reservoir uncertainty.

## 1.3 Surrogate Modeling as a Solution

To overcome this computational hurdle, the concept of surrogate modeling has gained significant traction. A surrogate model (or proxy model) is a computationally cheap approximation of the full-physics numerical simulator. The goal is to create a model that can rapidly produce outputs that are sufficiently accurate for the task at hand, enabling the execution of workflows that would otherwise be intractable.

Historically, methods like Response Surface Methodology (RSM) have been used, where polynomial functions are fitted to a small number of simulation results. While fast, these methods often struggle to capture the highly non-linear behavior of reservoir fluid flow. In recent years, the rapid advancement of machine learning and deep learning has introduced a paradigm shift. Data-driven techniques, particularly deep neural networks, have demonstrated a remarkable ability to learn complex, high-dimensional mappings from inputs (e.g., geological properties) to outputs (e.g., production profiles or pressure maps) [Cite a review paper on ML in petroleum engineering].

Of course. That's an excellent suggestion. Providing that context makes the choice of PINO much clearer and demonstrates a deeper understanding of the landscape of scientific machine learning. It builds the argument from the ground up.

## 1.4 From Neural Networks to Physics-Informed Neural Operators

While standard deep learning models have shown promise, their fundamental architecture poses limitations when applied to scientific problems governed by PDEs. A traditional neural network, such as a Convolutional Neural Network (CNN), learns a mapping between finite-dimensional Euclidean spaces (e.g., from one grid of pixels to another). If the resolution or discretization of the input data changes, the network must be retrained. This "mesh-dependent" nature is a significant drawback for scientific computing, where we often want to evaluate solutions on different grids.

To address this, the concept of **Neural Operators** has emerged as a more powerful and appropriate framework. Unlike traditional networks, a Neural Operator is designed to learn mappings directly between infinite-dimensional function spaces. It approximates an operator—a mapping from one function to another—making it inherently mesh-independent. Once trained on data from one discretization, a Neural Operator can evaluate on data with a different resolution without retraining, a crucial property for robust surrogate modeling.

Several architectures have been proposed to implement this concept. Two of the most prominent are:

* **DeepONet (Deep Operator Network):** This architecture approximates the operator using two separate neural networks: a "branch" net that encodes the input function at a fixed set of sensor locations, and a "trunk" net that takes as input the coordinates where the output function is to be evaluated. The outputs of these two networks are then combined (typically via a dot product) to produce the final prediction. DeepONet is very general but can be computationally intensive to train and query. [Cite the original DeepONet paper, Lu et al., 2021].

* **Fourier Neural Operator (FNO):** FNO takes a different approach by parameterizing the integral kernel of the operator directly in Fourier space. It leverages the convolution theorem, performing the global convolution operation efficiently as a pointwise multiplication in the frequency domain using the Fast Fourier Transform (FFT). By filtering out high-frequency modes, the FNO can effectively and efficiently learn the mapping from an input function to an output function. Its architecture is particularly well-suited for problems where spectral methods are effective. [Cite the original FNO paper, Li et al., 2020].

The **Physics-Informed Neural Operator (PINO)** represents the next logical evolution, specifically by augmenting the FNO framework with physical knowledge. While FNO learns the operator purely from data, PINO enriches the training process by adding a physics-based penalty to the loss function. In addition to the standard data-driven term that minimizes the mismatch with simulation results, a second "physics-loss" term is introduced. This term penalizes any deviation from the governing physical laws (the PDEs), which is calculated at arbitrary points in the domain using automatic differentiation.

By constraining the model with physics, PINO offers a powerful value proposition over a standard FNO:
* **Improved Data Efficiency:** The model requires less training data to achieve high accuracy because the physics provides a strong learning signal.
* **Enhanced Generalization:** Predictions are more robust and accurate for unseen inputs, as the model has learned the underlying structure of the problem, not just a data-driven correlation.
* **Physical Plausibility:** The model is explicitly guided to produce solutions that respect the fundamental laws of fluid dynamics, preventing non-physical artifacts.

Given its architectural efficiency and its ability to embed physical constraints, PINO was selected as the most suitable method for this project.

## 1.5 Project Objectives

This project aims to investigate the application of PINO for developing a fast and accurate surrogate model for two-phase flow in a porous medium. The primary goal is to demonstrate a proof-of-concept workflow, from data preparation to model validation. The specific objectives are:

1.  To develop a robust Python-based data processing pipeline capable of parsing an ensemble of raw binary simulation results from Eclipse (`.UNRST` files) and structuring them into a format suitable for machine learning.
2.  To implement and train a PINO model, using the PyTorch library, to learn the mapping from static reservoir permeability fields to the corresponding dynamic time-series of pressure and oil saturation fields.
3.  To rigorously evaluate the performance of the trained PINO surrogate by comparing its predictions against the ground-truth results from the Eclipse simulator on a set of unseen test cases.
4.  To critically analyze the model's accuracy, computational speed-up, and the specific benefits derived from the physics-informed training paradigm, thereby assessing the suitability of PINO for practical reservoir engineering applications.

Excellent points. These details will significantly increase the technical depth and accuracy of the report. Incorporating the full set of input features and discussing the scaling choices are particularly important for demonstrating a thorough understanding of the practical aspects of the project.

Here is the revised and expanded version of Chapter 2, "Formalism and Methods," incorporating all of your feedback.

***

# 2. Formalism and Methods

This chapter details the theoretical and practical methodologies employed in this project. We begin by presenting the governing physical equations for three-phase flow in porous media, including the boundary and initial conditions that define the problem. We then discuss the numerical simulation approach used to generate the training data and detail the comprehensive data preparation pipeline, which handles a multi-component input tensor and selective feature scaling. Finally, we provide a detailed overview of the Fourier Neural Operator and the specific mathematical formulation of the Physics-Informed Neural Operator (PINO) architecture used to create the surrogate model.

## 2.1 Governing Equations and Problem Definition

### 2.1.1 Three-Phase Flow Equations
The simultaneous flow of three immiscible and compressible fluids (oil, water, and gas, denoted by subscripts $o, w, g$) through a porous medium is described by a set of coupled, non-linear partial differential equations (PDEs). These equations are derived by combining the principle of mass conservation for each phase with Darcy's Law, which describes fluid motion in a porous medium.

The general mass conservation equation for a phase $\alpha$ is:

$$\frac{\partial}{\partial t} (\phi \rho_{\alpha} S_{\alpha}) + \nabla \cdot (\rho_{\alpha} \mathbf{u}_{\alpha}) - q_{\alpha} = 0$$

where $\mathbf{u}_{\alpha}$ is the Darcy velocity vector for phase $\alpha$, given by:

$$\mathbf{u}_{\alpha} = - \frac{k_{r\alpha}}{\mu_{\alpha}} \mathbf{K} (\nabla P_{\alpha} - \rho_{\alpha} g \nabla z)$$

For simplicity in this formulation, we consider fluid densities ($\rho_{\alpha}$) at reservoir conditions. In a full-fledged simulator, these terms are often expressed using a formation volume factor ($B_\alpha$) and density at standard surface conditions ($\rho_{\alpha, sc}$), such that $\rho_{\alpha} = \rho_{\alpha, sc} / B_\alpha$. The formation volume factor accounts for changes in fluid volume due to pressure and temperature differences between the reservoir and the surface. While omitted here for clarity, its effects are implicitly captured in the ground-truth data generated by the Eclipse simulator.

To close this system of three PDEs, several auxiliary equations are required:

1.  **Saturation Constraint:** The saturations of all phases must sum to unity in the pore space. Water ($S_w$) and gas ($S_g$) saturations are treated as primary variables, while the oil saturation ($S_o$) is calculated directly from this constraint:
    $$S_o = 1 - S_w - S_g$$
2.  **Capillary Pressure:** The pressure differences between the phases are non-linear functions of the saturations:
    $$P_{cow}(S_w) = P_o - P_w \quad \text{and} \quad P_{cgo}(S_g) = P_g - P_o$$
3.  **Relative Permeability:** The relative permeability of each phase ($k_{ro}, k_{rw}, k_{rg}$) is a non-linear function of the phase saturations.
4.  **Fluid Properties (PVT):** Fluid densities ($\rho_{\alpha}$) and viscosities ($\mu_{\alpha}$) are themselves functions of pressure.

### 2.1.2 Initial and Boundary Conditions
To form a well-posed problem, the system of PDEs must be supplemented with initial and boundary conditions.
* **Initial Conditions (IC):** The state of the reservoir must be defined at time $t=0$. This includes the initial pressure field, $P(x, y, z, t=0)$, and the initial saturation fields, $S_w(x, y, z, t=0)$ and $S_g(x, y, z, t=0)$.
* **Boundary Conditions (BC):** These define how the system behaves at the external boundaries of the reservoir model. For this project, **no-flow** (or Neumann) boundary conditions are assumed, meaning there is no fluid flux across the outer faces of the grid. This is a common assumption for modeling closed or very large reservoir systems and is mathematically expressed as:
    $$\mathbf{u}_{\alpha} \cdot \mathbf{n} = 0 \quad \text{at the boundary}$$
    where $\mathbf{n}$ is the normal vector to the boundary surface.

## 2.2 Numerical Simulation and Data Preparation

The ground-truth data for this project was generated using the Eclipse reservoir simulator. An ensemble of simulations was run, where each member of the ensemble represented a different geological realization.

### 2.2.1 Numerical Discretization and Simulation
Eclipse solves the governing equations using a Finite Difference Method (FDM), discretizing the reservoir into a structured grid and the time domain into discrete steps. This transforms the continuous PDEs into a large system of non-linear algebraic equations, which are solved at each time step to advance the simulation. The simulation workflow relies on parsing various input files (e.g., `.DATA`, `.INC`) that define the grid, fluid properties, and geological features like faults, and it produces binary output files (`.UNRST`, `.UNSMRY`) containing the solution. For this work, we primarily use the high-resolution restart (`.UNRST`) files which store the full 3D field data for all properties at specified timesteps.

### 2.2.2 Feature Engineering and Scaling
A key task was to structure the raw simulation data into a format suitable for supervised learning.

**Input Features (`a(x)`):** The input to our neural operator is not a single property, but a multi-channel 3D tensor representing the static properties of the reservoir. For each grid cell, the input vector consists of five features:
1.  **Permeability (K):** The absolute permeability field.
2.  **Porosity ($\phi$):** The rock porosity field.
3.  **Fault Transmissibility Multiplier (FLTM):** A scalar field, typically 1.0, that is reduced at fault locations to model sealing or partially sealing faults.
4.  **Initial Water Saturation ($S_{w,init}$):** The water saturation field at $t=0$.
5.  **Initial Pressure ($P_{init}$):** The pressure field at $t=0$.

**Output Targets (`u(x,t)`):** The desired output is the dynamic, 4D fields of pressure ($P$) and saturations ($S_w, S_g$) over time.

**Feature Scaling:** Appropriate scaling is crucial for stable and efficient neural network training. A mixed scaling strategy was adopted:
* **Permeability:** Due to its log-normal distribution spanning several orders of magnitude, permeability was scaled using a **log-transform followed by z-score standardization**:
    $$K_{scaled} = \frac{\log(K) - \mu_{\log K}}{\sigma_{\log K}}$$
* **Pressure and FLTM:** These fields were scaled using standard **Min-Max normalization** to the [0, 1] range.
* **Porosity ($\phi$) and Water Saturation ($S_w$):** These variables were **not scaled**, as their physical values already lie within the natural range of [0, 1]. This choice has pros and cons:
    * **Pros:** It preserves the direct physical meaning of the output (e.g., a predicted value of 0.3 corresponds exactly to 30% saturation), avoids introducing minor numerical shifts from the scaling/unscaling process, and simplifies the data pipeline.
    * **Cons:** The unscaled features will have a different statistical distribution (e.g., mean and variance will not be 0 and 1) compared to the standardized features. In theory, this could slightly unbalance the gradient updates in the early stages of training. However, for variables in a tight, well-defined range like [0, 1], this effect is often negligible and outweighed by the benefits of interpretability and simplicity.

All scaling parameters (min/max values, means, standard deviations) were computed from the training set and stored for inverse-transforming the model's outputs back to their physical units for validation and analysis.

## 2.3 From Neural Networks to Neural Operators: A Formal View

Before detailing the specifics of the PINO architecture, it is crucial to formally distinguish the Neural Operator framework from that of traditional deep neural networks. This distinction highlights a fundamental shift in machine learning paradigms, moving from simple function approximation to learning operators between infinite-dimensional function spaces, a concept perfectly suited for solving PDEs.

### 2.3.1 The Limitation of Standard Neural Networks in Scientific Computing
A standard deep neural network, such as a Multi-Layer Perceptron (MLP) or a Convolutional Neural Network (CNN), is a powerful function approximator. However, it is fundamentally designed to learn mappings between finite-dimensional Euclidean spaces. Mathematically, such a network $\mathcal{N}$ learns a function:

$$\mathcal{N}: \mathbb{R}^{d_{in}} \to \mathbb{R}^{d_{out}}$$

When we apply this to a problem governed by PDEs, we are forced to work with a specific discretization of the continuous functions. For an input function $a(x)$ and an output function $u(x)$ defined over a domain $\Omega$, we represent them on a grid with $N$ points. The network then learns a mapping between the grid point values:

$$\mathcal{N}_{\theta}: \mathbb{R}^{N \times c_{in}} \to \mathbb{R}^{N \times c_{out}}$$

where $c_{in}$ and $c_{out}$ are the number of input and output channels, respectively. The core limitation is that the learned parameters $\theta$ of the network are intrinsically tied to the specific discretization $N$. If the resolution of the grid changes (e.g., from ($100 \times 100 \times 20$) to ($50 \times 50 \times 20$)), the input and output dimensions change, and the network cannot be applied. A new network must be designed and retrained from scratch. This "mesh-dependent" nature makes standard neural networks brittle and inefficient as general-purpose PDE solvers.

### 2.3.2 The Neural Operator Framework
A Neural Operator addresses this limitation by working in a different paradigm. Instead of learning a mapping between fixed-dimensional vectors, a Neural Operator learns a mapping directly between function spaces. It aims to approximate an **operator** $\mathcal{G}$, which is itself a function that takes a function as input and returns another function as output:

$$\mathcal{G}: \mathcal{A}(\Omega) \to \mathcal{U}(\Omega)$$

Here, $\mathcal{A}$ and $\mathcal{U}$ are infinite-dimensional **Banach spaces** of functions defined over the domain $\Omega$. This mathematical formalism is not just abstract; it has critical practical implications for why the framework is so robust for physical problems.

#### The Practical Meaning of Banach Spaces
Stating that our inputs and outputs are functions in Banach spaces is a rigorous way of saying that our physical fields are **well-behaved and measurable**. It provides the theoretical safety net needed for the operator concept to work. A Banach space has two key properties:

1.  **It possesses a norm (`||.||`).** A norm is a function that assigns a non-negative "length" or "size" to every function in the space. For our physical fields, this is intuitive. For example, the **input** permeability field, `K(x)`, can have its "size" measured by the L-infinity norm ($L^{\infty}$), which is simply the maximum permeability value. The **output** pressure field, `P(x, t)`, could be measured with a similar norm, or perhaps the L2-norm ($L^2$), which relates to the total potential energy. This ability to measure size is what allows us to define a meaningful loss function. When we calculate `||u_pred - u_true||`, we are using the norm to measure the "distance" (the size of the difference) between the predicted and true functions. Without a norm, we could not quantify error.

2.  **The space is "complete".** This is a more abstract but crucial property. "Completeness" guarantees that if we have a sequence of functions that are getting progressively closer and closer to each other (a "Cauchy sequence"), their limit is **guaranteed to also be a function within that same space**. In practice, this means that as our PINO model trains and produces a sequence of improving predictions ($u_{pred,1}, u_{pred,2}, ...$), the ideal function it converges to—the final prediction $u_{final}$—will still be a valid, well-behaved pressure field of the same kind. It won't suddenly become infinitely spiky or "disappear" from the space of possible solutions. This guarantees a stable endpoint for our optimization process.

#### When This Framework Might Not Apply
Most physically meaningful fields fit this description, but it's useful to consider cases where they would not, to understand the boundaries of the method:
* **Categorical Inputs:** Imagine an input field that isn't a continuous number like permeability, but a **categorical rock type map** (e.g., 'Sand', 'Shale'). You cannot directly define a norm or distance between these discrete labels. Such a map does not live in a Banach space of functions. In practice, we would handle this by converting the map into numerical fields using one-hot encoding.
* **Functions with Severe Discontinuities:** An idealized model of a well as a "needle" of zero radius would create a pressure solution with a singularity (infinite pressure or drawdown). A function containing an infinity point is not part of many standard Banach spaces because its norm would be infinite.
* **Input as a Statistical Description:** If the input was not a single permeability field, but the abstract statistical parameters that *describe* a family of fields (e.g., mean, variance), these few numbers do not form a function over a spatial domain.

For the vast majority of practical reservoir simulation problems, where inputs and outputs are physical fields with finite values represented on a grid, the Banach space framework is a natural and robust mathematical choice.

The overall goal is to train a single parametric model $\mathcal{G}_{\theta}$ that approximates $\mathcal{G}$. Once trained, $\mathcal{G}_{\theta}$ can take any function $a \in \mathcal{A}$ represented by any discretization as input and produce a corresponding solution function $u \in \mathcal{U}$, which can also be evaluated at any point in the domain. This property is known as **mesh-independence** or **zero-shot super-resolution**.

### 2.3.3 The Power of the Integral Operator Formulation

To understand why an integral formulation is so powerful, it is helpful to first consider the problem of solving a PDE from the perspective of operator calculus. A general linear PDE can be expressed compactly as:

$$\mathcal{D} u(x) = f(x)$$

where $\mathcal{D}$ is a differential operator, $f(x)$ is a known source function, and $u(x)$ is the unknown solution function. For a relevant example from our context, consider the steady-state, single-phase pressure equation in a reservoir. Combining Darcy's law (including gravity) and the mass conservation equation gives:

$$-\nabla \cdot \left( \frac{\mathbf{K}(x)}{\mu} (\nabla P(x) - \rho g \nabla z) \right) = q(x)$$

Here, the differential operator is $\mathcal{D} = -\nabla \cdot \left( \frac{\mathbf{K}(x)}{\mu} \nabla \right)$, which acts on the pressure field $P(x)$. The source function $f(x)$ can be seen as an "effective" source term that includes both the explicit well terms, $q(x)$, and the implicit gravitational effects. The solution function $u(x)$ is the resulting pressure field $P(x)$.

From this perspective, "solving" the PDE is equivalent to finding the **inverse operator**, $\mathcal{D}^{-1}$, and applying it to the effective source function. This inverse operator would directly map the well rates and gravitational forces to the corresponding pressure field:

$$P(x) = \mathcal{D}^{-1} f(x)$$

For a vast number of physical systems described by differential equations, this abstract inverse operator $\mathcal{D}^{-1}$ takes the concrete mathematical form of an **integral operator**. The solution $P(x)$ can be expressed in terms of the input function $f(x)$ using a specific integral kernel known as the Green's function, $G(x,y)$:

$$P(x) = \int_{\Omega} G(x, y) f(y) dy$$

Comparing the two equations, we see that the integral with the Green's function *is* the inverse operator. The Green's function $G(x,y)$ implicitly encodes the physics of the differential operator $\mathcal{D}$ (including the geology via $\mathbf{K}(x)$) and its boundary conditions. However, for the complex, non-linear, and time-dependent systems in multiphase reservoir simulation, an analytical form for $G(x,y)$ is rarely, if ever, known.

The central idea behind modern Neural Operators, like the FNO, is to directly parameterize and learn a kernel function $\kappa_{\theta}(x, y)$ that approximates the action of this unknown integral operator. The core of a neural operator layer can be expressed as an update step of the form:

$$(v_{t+1})(x) = \sigma \left( W v_t(x) + \int_{\Omega} \kappa_{\theta}(x, y, v_t(y)) v_t(y) dy \right)$$

where $v_t$ is the function representation at layer $t$, $W$ is a local linear transformation (like a 1x1 convolution), and $\sigma$ is a non-linear activation function. The integral term is key:
* **It is a global operator:** The value of the output at a single point $x$ depends on the value of the input function $v_t(y)$ over the entire domain $\Omega$. This is essential for modeling PDEs, where local changes can have non-local effects.
* **It is mesh-independent:** The learned kernel $\kappa_{\theta}$ is a continuous representation that is not tied to a specific grid. The integral can be approximated using any discretization of the domain $\Omega$.

By stacking these layers, the network can learn highly non-linear operators. The FNO, as we will see, provides a particularly efficient way to compute this integral operator for all points $x$ simultaneously using the convolution theorem and the Fast Fourier Transform. This elegant formulation, which combines generality, physical intuition, and computational efficiency, makes Neural Operators a superior framework for surrogate modeling of physical systems.

## 2.4 Physics-Informed Neural Operator (PINO) Architecture

The PINO framework, built upon the FNO, was selected for its mesh-independent properties and its ability to embed physical laws directly into the training process.

### 2.4.1 Fourier Neural Operator (FNO)
There are two main motivations to use Fourier transformation. First, it’s fast. A full standard integration of n points has complexity $O(n^2)$, while convolution via Fourier transform is quasilinear. Second, it’s efficient. The inputs and outputs of PDEs are continuous functions. So it’s usually more efficient to represent them in Fourier space.

The power of the FNO comes from its efficient implementation of the global integral operator we discussed previously. Let's revisit the general form of that operator:

$$(v_{t+1})(x) = \sigma \left( W v_t(x) + \int_{\Omega} \kappa_{\theta}(x, y) v_t(y) dy \right)$$

The challenge lies in the integral term. The kernel $\kappa_{\theta}(x, y)$ is a general function of two variables, $x$ and $y$. Computing this integral directly is computationally prohibitive. This is where the FNO employs an elegant and powerful simplification.

#### The Convolution "Trick" and Translational Invariance

The **Convolution Theorem** provides a highly efficient way to compute a specific type of integral, a **convolution**. A convolution between a kernel $k$ and a function $v$ is defined as:

$$(k * v)(x) = \int_{\Omega} k(x-y) v(y) dy$$

Notice the critical difference in the kernel: in a convolution, the kernel $k(x-y)$ does not depend on the absolute positions $x$ and $y$ independently, but only on their relative displacement vector, $x-y$. This property is known as **translational invariance** (or stationarity). It implies that the influence of an input at point $y$ on an output at point $x$ is the same for any pair of points with the same separation vector.

The general integral operator with kernel $\kappa_{\theta}(x, y)$ is *not* a convolution and does not necessarily have this property. **The core architectural assumption of the FNO is to restrict the learned kernel $\kappa_{\theta}$ to be translationally invariant.** That is, it constrains the kernel such that $\kappa_{\theta}(x, y) = \kappa_{\theta}(x-y)$.

By making this simplifying assumption, the general integral becomes a convolution. Now, the Convolution Theorem can be applied, which states that a convolution in the spatial domain is equivalent to a simple, element-wise multiplication in the frequency domain:

$$\mathcal{F}(k * v) = \mathcal{F}(k) \cdot \mathcal{F}(v)$$

This is the "trick" that gives the FNO its power and efficiency. Instead of learning the complex kernel $\kappa_{\theta}(x,y)$ and performing a costly integral, the FNO learns the Fourier transform of the kernel, $\mathcal{F}(\kappa_{\theta})$, and performs a fast element-wise multiplication in the frequency domain.

An FNO model architecture, introduced in [Zongyi Li], is shown below.

<p align="center">
<img src="fourier_neural_operator_zongyi_li.png" width="800" height="300">
</p>


1.  **Lifting/projection ($P$):** The input function $a(x)$ is lifted to a higher-dimensional channel space, $v_0(x)$, using a local linear transformation.
2.  **Fourier Transform ($\mathcal{F}$):** The function $v_0(x)$ is transformed to the frequency domain using the Fast Fourier Transform (FFT), yielding $\mathcal{F}(v_0)$.
3.  **Linear Transform in Fourier Space ($R$):** This is the core learning step. A filter, represented by a complex-valued weight matrix $R_{\theta}$, is multiplied element-wise with the lower-frequency modes of $\mathcal{F}(v_0)$. High-frequency modes are truncated (set to zero). This single multiplication efficiently performs the global convolution.
4.  **Inverse Fourier Transform ($\mathcal{F}^{-1}$):** The result is transformed back to the spatial domain using the inverse FFT.
5.  **Local Transform and Combination ($W$):** The result from the Fourier path is added to another local linear transformation of the lifted function ($Wv_0(x)$) via a skip connection.
6.  **Activation:** The combined result is passed through a non-linear activation function $\sigma$ (e.g., GELU) to produce the output of the layer. Notice the activation functions are applied on the spatial domain. They help to recover the Higher frequency modes and non-periodic boundary which are left out in the Fourier layers. Therefore it’s necessary to do the Fourier transform and its inverse at each layer.

The overall update from one layer to the next is: $v_{t+1}(x) = \sigma(\mathcal{F}^{-1}(R_{\theta} \cdot \mathcal{F}(v_t)) + Wv_t)$. Stacking these layers allows the FNO to approximate highly non-linear and non-local operators efficiently.

Let's walk through the steps of a single FNO layer using one of our primary input features, the **permeability map** `K(x,y,z)`, as a concrete example.

The goal of the layer is to transform this input permeability map into a new feature map where every point's value is influenced by *all* points in the original map, thereby capturing global physical dependencies.

**Step 1: Lifting to a Higher-Dimensional Representation**

The input permeability map is a single-channel 3D grid of values. The first step, **lifting**, uses a simple linear transformation (like a 1x1 convolution) to project this single channel into a multi-channel feature map, say with 32 channels. Let's call this lifted function $v_0(x,y,z)$. This gives the network a richer, higher-dimensional space to learn complex features beyond the raw permeability value at each point.

**Step 2: Transforming to the Frequency Domain (FFT)**

Next, the network applies the **Fast Fourier Transform (FFT)** to each of the 32 channels of $v_0(x,y,z)$. The FFT decomposes each channel's spatial map into its constituent frequencies.
* **Low frequencies** represent the large-scale, smooth trends in the permeability field (e.g., the orientation of a large channel or a gradual change in rock quality).
* **High frequencies** represent the small-scale, sharp, and jagged variations (e.g., fine-scale heterogeneity or noise).
The output, $\mathcal{F}(v_0)$, is now a representation of the permeability field in the frequency domain, where each point corresponds to the amplitude and phase of a specific frequency.

**Step 3: Learning in the Frequency Domain (Filtering)**

This is the core learning step of the FNO and what makes it so efficient. Instead of performing a costly convolution in the spatial domain, the network performs a simple, element-wise multiplication in the frequency domain.

First, the frequency representation is **truncated**. Only a limited number of low-frequency modes (e.g., the lowest 12x12 modes) are kept, while the high-frequency modes are discarded (zeroed out). This acts as a form of regularization, based on the physical assumption that the solution (e.g., the pressure field) is generally smoother than the highly heterogeneous input permeability field.

Then, a **learned filter**, represented by a complex-valued weight tensor $R_{\theta}$, is multiplied element-wise with these kept low-frequency modes. This learned filter is the key parameter of the layer. By adjusting the values in $R_{\theta}$, the network learns how to modify the amplitudes and phases of the dominant large-scale trends in the permeability field to best predict the output pressure and saturation.

**Step 4: Transforming Back to the Spatial Domain (IFFT)**

The filtered frequency representation is then transformed back into the spatial domain using the **Inverse Fast Fourier Transform (IFFT)**. The result is a new 32-channel feature map. Because the filtering operation happened in the frequency domain, every point in this new map has been influenced by every point in the original input map. It is the result of the global convolution.

**Step 5: Combining Global and Local Information**

The FNO architecture includes a **skip connection**. The original lifted function, $v_0(x,y,z)$, is also passed through another simple, local linear transformation $W$ (another 1x1 convolution). This purely local information is then added to the globally convoluted map produced by the Fourier path. This allows the network to combine long-range dependencies (captured via the FFT) with simple local features, which is crucial for modeling complex physical phenomena.

**Step 6: Applying Non-Linearity**

Finally, the combined feature map is passed through a non-linear **activation function** (such as GELU). This is a critical step that allows the network, when layers are stacked, to learn the highly non-linear relationships between permeability, pressure, and saturation.

The output of this entire process is a new, transformed feature map, $v_1(x,y,z)$, which then serves as the input to the next FNO layer in the network.

### 2.4.2 From Continuous Integrals to Discrete Computation in 3D

So far, our discussion of operators has been in the continuous domain, dealing with functions like $v(\mathbf{x})$ and integrals over a domain $\Omega$. However, a computer does not store continuous functions. It stores discrete arrays of numbers. This section explains how we bridge this gap for our 3D model and handle the practical requirements of the computational methods used.

#### Discretization of Continuous Functions

A physical field in our 3D reservoir model, such as the permeability, is a continuous function $K(\mathbf{x}) = K(x,y,z)$, where a value exists at every coordinate point $\mathbf{x} = (x,y,z)$. To work with this on a computer, we must **discretize** it. We overlay a 3D grid on the domain and store the value of the function at the center of each grid cell (or block). Our continuous function is therefore represented by a discrete 3D array (a tensor), `K[i, j, k]`, where `i`, `j`, and `k` are the integer indices of the grid cell along each axis.

#### The Discrete Convolution and the FFT

With our functions represented as discrete 3D arrays, the continuous integral in the operator equation must be replaced by a numerical approximation. A continuous integral is the limit of a sum, so we can approximate it with a finite, weighted sum over all grid cells. The integral involving a kernel $k(\mathbf{x}-\mathbf{y})$:

$$\int_{\Omega} k(\mathbf{x}-\mathbf{y}) v(\mathbf{y}) d\mathbf{y}$$

is approximated by the discrete 3D summation, where $\mathbf{i}=(i_x, i_y, i_z)$ and $\mathbf{j}=(j_x, j_y, j_z)$ are multi-indices for the grid cells:

$$\sum_{\mathbf{j}} k[\mathbf{i}-\mathbf{j}] v[\mathbf{j}] \Delta V$$

Here, $\Delta V$ is the volume of a single grid cell. This sum is a **discrete 3D convolution**. While we could compute this sum directly, it would be extremely slow, requiring on the order of $N^2$ operations, where $N$ is the total number of cells. This is where the true power of the FNO's design comes into play. The **Fast Fourier Transform (FFT)** provides a much more efficient path via the Discrete Convolution Theorem, which holds true in any number of dimensions.

#### The Challenge of Non-Periodic Physical Data

The validity of the FFT rests on a critical underlying assumption: **periodicity**. The 3D Discrete Fourier Transform inherently treats the finite 3D grid as if it were a single period of an infinitely repeating lattice. This is equivalent to treating the domain as a **3D torus**, where opposite faces of the model cube are seamlessly connected (right face to left, top to bottom, front to back).

This periodicity assumption creates a significant challenge for real-world geological models, which are almost never periodic. Consider a realistic **3D permeability model**:
* A high-permeability river channel might enter from the left face of the model and terminate in the center.
* A sealing fault might run vertically through one side of the model.

When the FFT processes this model, it assumes the domain wraps around. The channel that stops in the middle would create an artificial, abrupt discontinuity where it meets the "wraparound" boundary. These non-periodic features introduce strong, artificial high-frequency components into the frequency spectrum, an artifact known as **spectral leakage**, which can corrupt the learned kernel and lead to inaccurate results.

#### The Solution: Zero-Padding

The standard technique to mitigate this issue is **zero-padding**. Before applying the FFT, we expand the domain of our 3D data by surrounding it with a border of zeros. For example, a 3D permeability grid of size `64x64x20` might be embedded in the center of a larger `96x96x32` grid filled with zeros.

This padding serves a crucial purpose: it pushes the physically meaningful, non-periodic parts of the data away from the "wraparound" edges of the computational domain. The abrupt jump from the edge of a geological feature to the edge of the grid now becomes a much smoother transition to a large region of zeros. This significantly reduces the artificial high-frequency artifacts, allowing the FFT to compute a much more accurate frequency representation of the actual, non-periodic 3D data.

In summary, while the FFT is an extraordinarily efficient numerical integration scheme for convolutions in 3D, we must respect its underlying assumption of periodicity. By using zero-padding, we adapt our non-periodic physical data to this requirement, ensuring a more accurate and stable computation within the FNO architecture.

### 2.4.3 The PINO Loss Function
The PINO enhances the FNO by incorporating physical laws into its training via a composite loss function.

**1. Data Mismatch Loss ($L_{data}$):** This is the standard supervised loss, forcing the model's predictions to match the Eclipse simulation data on the training set $S_{data}$. We use the Mean Squared Error (MSE):
$$L_{data} = \frac{1}{|S_{data}|} \sum_{(a, u_{true}) \in S_{data}} || u_{pred}(a) - u_{true} ||^2_{2}$$
where $u_{pred}(a)$ is the PINO output for the multi-channel static input tensor $a$.

**2. Physics Residual Loss ($L_{phys}$):** This term enforces the governing PDEs from Section 2.1.1. Let the PDE system be represented by the operator $\mathcal{N}(u(x,t); a(x)) = 0$. The physics loss penalizes non-zero residuals at a set of collocation points $S_{phys}$ randomly sampled across the spatio-temporal domain:
$$L_{phys} = \frac{1}{|S_{phys}|} \sum_{(x, t) \in S_{phys}} || \mathcal{N}(u_{pred}(x,t); a(x)) ||^2_{2}$$
Crucially, the derivatives needed to compute the residual (e.g., $\frac{\partial P}{\partial t}, \nabla P$) are calculated from the network's output using **automatic differentiation**, a core feature of PyTorch.

**3. Total Loss Function:** The final loss is a weighted sum of the data and physics losses, balanced by hyperparameters $\lambda_{data}$ and $\lambda_{phys}$:
$$L_{total} = \lambda_{data} L_{data} + \lambda_{phys} L_{phys}$$
This formulation allows the model to learn from sparse or noisy data while ensuring its predictions remain physically plausible and generalize better to unseen scenarios.

Of course. Here is a detailed report on the data preparation pipeline, as implemented in the `prepare_training_data.py` and `eclipse_reader.py` scripts.

***

# **3. Code, Implementation, and Testing**
*(Target: ~3-4 pages | Grading Points: 20)*
* **3.1 Development Environment:** List of key software (Python) and libraries (PyTorch).
* **3.2 Implementation of Data Pipeline:** Commented Python code snippets for reading `.UNRST` files and creating PyTorch tensors.
* **3.3 PINO Model Implementation:** Code defining the PINO model, the physics-loss function using `torch.autograd`, and the training loop algorithm.
* **3.4 Training and Validation Protocol:** Description of the train/validation/test split and key hyperparameters.
* **3.5 Testing and Benchmarking:** Procedure for testing on unseen data and definition of evaluation metrics (Relative L2 error for accuracy, wall-clock time for performance).
Of course. Here is a detailed report on the data preparation pipeline, as implemented in the `prepare_training_data.py` and `eclipse_reader.py` scripts.

***

## Report: PINO Surrogate Model Data Preparation Pipeline

**Version:** 1.0
**Date:** July 26, 2024
**Author:** Gemini AI Assistant

### 1. Introduction and Pipeline Overview

The successful application of machine learning to complex physical domains, such as reservoir simulation, hinges on a robust and meticulously designed data preparation pipeline. This document details the architecture and functionality of the data preparation pipeline for the Physics-Informed Neural Operator (PINO) surrogate model project. The pipeline's primary objective is to process raw output files from the Eclipse reservoir simulator and transform them into structured, normalized, and model-ready datasets.

The pipeline is engineered to produce two distinct datasets tailored for different modeling components:

1.  **Grid-Based Dataset (`X_data1`):** A comprehensive, 5-dimensional dataset containing static and dynamic reservoir properties (e.g., permeability, pressure, saturation) across the entire grid and over multiple timesteps. This dataset is the primary input for the main PINO model, which learns the spatio-temporal dynamics of the reservoir.
2.  **Well-Centric Feature Dataset (`X_data2`):** A tabular dataset designed for a Peaceman well model. It contains feature vectors for each timestep, summarizing well-bore conditions and global properties, which are used to predict well-specific production rates.

#### 1.1. Core Technologies

The pipeline is built using standard, high-performance Python libraries:
*   **NumPy:** For efficient, n-dimensional array computation, forming the backbone of the data structures.
*   **Pandas:** Used for handling tabular data, particularly when parsing well summary files.
*   **Resdata & Res2df:** Specialized, domain-specific libraries that provide the critical interface for reading and interpreting proprietary Eclipse binary file formats (`.EGRID`, `.INIT`, `.UNRST`, `.SMSPEC`).
*   **Pathlib & OS:** For robust, cross-platform file and path manipulation.
*   **Logging:** For detailed, multi-level logging to both console and file, enabling effective debugging and monitoring.
*   **Pickle & Gzip:** For serializing the final NumPy-based data structures into compressed binary files for efficient storage and loading.

#### 1.2. High-Level Pipeline Architecture

The pipeline follows a clear, hierarchical process, orchestrated by the `prepare_training_data.py` script and leveraging the specialized `eclipse_reader.py` for data extraction.

### 2. Core Components and Data Flow

The pipeline's logic is cleanly separated into two main scripts: an orchestrator that manages the overall workflow and a specialized reader that handles the low-level details of file parsing.

#### 2.1. The Orchestrator: `prepare_training_data.py`

This script serves as the main entry point and controls the entire data preparation workflow.

*   **`prepare_ensemble_training_data()` Function:** This is the master function.
    *   **Discovery:** It begins by scanning a specified `ensemble_dir` for simulation folders, identified by a `realization_pattern` (e.g., "realization-*"). It can process a subset of these by setting `max_realizations`.
    *   **Configuration:** It accepts critical parameters that define the structure of the output data, including grid dimensions (`nx`, `ny`, `nz`) and the exact timesteps to be extracted (`steppi`, `steppi_indices`).
    *   **Iteration:** It loops through each discovered realization directory, invoking `process_single_realization` to handle each one individually. Any realization that fails is logged and skipped, allowing the pipeline to continue robustly.
    *   **Aggregation:** As each realization is processed, its data (returned as a dictionary of NumPy arrays) is appended to a master dictionary, `all_data`.
    *   **Post-Processing:** After all realizations are processed, it orchestrates the final steps: data cleaning (`clean_dict_arrays`), feature scaling (`scale_dict_arrays`), and the construction of the separate Peaceman dataset (`build_peaceman_input_data`).
    *   **Serialization:** Finally, it saves the fully processed and scaled `X_data1` and `X_data2` dictionaries into gzipped pickle files, along with JSON files containing the scaling parameters.

*   **`process_single_realization()` Function:** This function is the workhorse for a single simulation case.
    *   **File Location:** It is responsible for locating all required files within a specific realization folder (e.g., `realization-1/`). This includes grid files (`.EGRID`), initialization files (`.INIT`), restart files (`.UNRST`), summary data files (`.DATA`), and include files like `ACTNUM.grdecl` and fault definitions.
    *   **Data Extraction:** It instantiates the `EclipseReader` and calls its methods to extract the raw data from these files. This includes static properties, initial conditions, and time-varying properties.
    *   **Data Assembly:** It assembles all extracted NumPy arrays into a single dictionary, using human-readable keys like `'permeability'`, `'Pressure'`, and `'Water_saturation'`.
    *   **Validation:** It performs critical validation by comparing the dimensions of the read data against the expected grid dimensions and comparing the number of active cells read from `ACTNUM.grdecl` with the number reported by the grid object itself, flagging potential inconsistencies.

#### 2.2. The Data Extractor: `eclipse_reader.py`

This script provides the `EclipseReader` class, a powerful abstraction layer over the `resdata` and `res2df` libraries, simplifying interaction with the complex Eclipse file formats.

*   **`EclipseReader` Class:** Encapsulates all functionality for reading specific file types.
    *   **Grid and Static Data Reading (`read_grid`, `read_init`):** It reads the `.EGRID` file to understand the grid's structure and dimensions, and the `.INIT` file to extract static rock properties like `PERMX` (permeability) and `PORO` (porosity).
    *   **Dynamic Data Reading (`open_restart_file`, `read_dynamic_steps`):** It manages the `.UNRST` file, which contains snapshots of the reservoir state at different times. The `read_dynamic_steps` method is crucial; it reads data for a specific list of report steps and correctly maps the data, which is often stored only for active cells, onto the full 3D grid.
    *   **Well and Rate Data Reading (`read_summary_rates`, `extract_well_production_data`, `read_well_types_from_schedule`):** This is a sophisticated set of functions. `read_well_types_from_schedule` parses the `SCHEDULE` file to determine if a well is a producer or injector. `read_summary_rates` uses `res2df` to parse the `.UNSMRY` files, which contain production/injection rates on a per-connection basis (e.g., `COPR:WELL_A:10,20,5`). It then intelligently maps these point-source rates onto a 4D grid `(time, nz, nx, ny)`, effectively converting sparse data into a dense format suitable for CNN-based models.
    *   **Fault Data Generation (`create_fault_multiplier_array`):** This method performs feature engineering by creating a new 3D grid property. It reads fault locations from a `grid.faults` file and their corresponding transmissibility multipliers from a `multflt.inc` file, then "paints" these multiplier values onto the appropriate cells of a 3D NumPy array.

***

### 3. Data Transformation and Feature Engineering

The raw data from Eclipse is far from model-ready. The pipeline performs several crucial transformation and feature engineering steps.

#### 3.1. Grid and Property Handling

*   **ACTNUM Processing (`load_actnum`):** Eclipse's `ACTNUM` files define which grid cells are active (1) or inactive (0). These files often use a compressed `count*value` format (e.g., `1000*1 500*0`). The `load_actnum` function parses this format, expands it into a flat array, and reshapes it into the correct 3D grid structure `(nz, nx, ny)`, respecting the Fortran memory layout used by Eclipse.

*   **Active-to-Full Grid Mapping (`map_active_to_full_grid`):** This is one of the most critical helper functions. Most properties in `.UNRST` and `.INIT` files are stored as 1D arrays containing values for active cells only. This function takes this 1D array and the `resdata` grid object (which knows the mapping from active index to `i,j,k` coordinates) and generates a dense 3D NumPy array, filling all inactive cells with a specified default value (usually 0.0). This creates the uniform, dense tensors required by deep learning frameworks.

#### 3.2. Feature Engineering for the Peaceman Model

*   **`build_peaceman_input_data()`:** This function demonstrates significant feature engineering. It pivots from the grid-centric view to a well-centric one. For each producer well at each timestep, it constructs a feature vector by:
    1.  Identifying all grid cells the well is connected to.
    2.  Averaging the physical properties (permeability, saturations) across these connection cells.
    3.  Concatenating these averaged properties for all producer wells.
    4.  Appending the global average reservoir pressure and a normalized timestep value.

    The result is a feature matrix `X` of shape `(num_realizations, num_timesteps, num_features)`, ready for a recurrent or feed-forward network to predict well rates.

#### 3.3. Final Cleaning and Scaling

*   **Data Cleaning (`clean_dict_arrays`):** Before scaling, the pipeline ensures data integrity. This function iterates through all data arrays, replacing non-finite values (`NaN`, `inf`) with 0.0, and clips physically impossible values (e.g., negative porosity) to zero. This prevents numerical errors during training.

*   **Data Scaling (`scale_dict_arrays`):** Neural networks train most effectively when input features are normalized. This function applies min-max scaling to all relevant properties, transforming their values to the `[0, 1]` range.
    *   The `min` and `max` values for each property (e.g., 'permeability', 'Pressure') are either calculated from the training data or loaded from an external configuration file.
    *   Critically, these scaling parameters are saved to a separate `.json` file. This allows the exact same transformation to be applied to validation or test data, and enables the inverse transformation to be applied to model predictions to return them to their original physical units.

***

### 4. Pipeline Outputs and Conclusion

#### 4.1. Output Files

The pipeline produces a set of self-contained, analysis-ready files:

1.  **`..._train.pkl.gz` (X_data1):** This is the primary dataset for the PINO model. It's a gzipped dictionary where keys are property names (`'permeability'`, `'Pressure'`, etc.) and values are 5D NumPy arrays of shape `(num_realizations, num_timesteps, nz, nx, ny)`. Static properties are repeated across the time dimension for compatibility, having a shape of `(num_realizations, 1, nz, nx, ny)`.

2.  **`..._peaceman.pkl.gz` (X_data2):** This is the dataset for the well model. It's a dictionary containing the feature matrix `X` of shape `(num_realizations, timesteps, features)` and a target matrix `Y` of shape `(num_realizations, timesteps, 3 * num_producers)` for oil, water, and gas rates.

3.  **`.scaling.json`:** A JSON file for each dataset, storing the `min` and `max` values used to scale each feature. This is essential for reproducibility and for using the trained model in practice.

#### 4.2. Conclusion

The data preparation pipeline is a critical and sophisticated component of the PINO surrogate modeling project. It successfully bridges the gap between the complex, often unwieldy output of a traditional physics simulator and the structured, clean, and normalized tensor formats required by modern deep learning models.

Through its modular design, with a clear separation between the high-level orchestrator and the low-level data reader, the pipeline is both robust and extensible. It performs significant data transformation and feature engineering, handling domain-specific complexities like active/inactive cells, compressed data formats, and the mapping of sparse well data onto a dense grid. The final outputs are self-contained, model-ready datasets that provide a solid foundation for training high-fidelity surrogate models for reservoir simulation.

***

## Nomenclature

| Symbol | Description | Unit |
| :--- | :--- | :--- |
| $a(x)$ | Input function to the neural operator (multi-channel tensor) | - |
| $\mathcal{A}(\Omega)$ | Banach space of input functions | - |
| $c_{in}, c_{out}$ | Number of input/output channels for discretized functions | dimensionless |
| $d_{in}, d_{out}$ | Dimensionality of input/output vectors for standard NNs | dimensionless |
| $\mathcal{F}$ | Fourier Transform operator | - |
| $\mathcal{F}^{-1}$ | Inverse Fourier Transform operator | - |
| $g$ | Acceleration due to gravity | m/s² |
| $\mathcal{G}$ | True solution operator mapping input functions to output functions | - |
| $\mathcal{G}_{\theta}$ | Neural operator approximation of $\mathcal{G}$ with parameters $\theta$ | - |
| $G(x,y)$ | Green's function for an integral operator | - |
| $\mathbf{K}$ | Absolute permeability tensor | m² (or mD) |
| $k_{r\alpha}$ | Relative permeability of phase $\alpha$ | dimensionless |
| $\kappa_{\theta}$ | Learned integral kernel in a neural operator layer | - |
| $L_{data}$ | Data mismatch loss term | - |
| $L_{phys}$ | Physics residual loss term | - |
| $\lambda$ | Loss weighting hyperparameter | dimensionless |
| $\mu_{\alpha}$ | Viscosity of phase $\alpha$ | Pa·s (or cP) |
| $N$ | Number of points in a spatial discretization | dimensionless |
| $\mathcal{N}(\cdot)$ | PDE residual operator for the physics loss | - |
| $\mathbf{n}$ | Outward unit normal vector at a boundary | dimensionless |
| $P_{\alpha}$ | Pressure of phase $\alpha$ (o, w, g) | Pa |
| $P_{c\alpha\beta}$ | Capillary pressure between phases $\alpha$ and $\beta$ | Pa |
| $\phi$ | Porosity | dimensionless |
| $q_{\alpha}$ | Source/sink term for phase $\alpha$ | kg/(m³·s) |
| $R_{\theta}$ | Learned linear transformation (filter) in Fourier space | - |
| $\rho_{\alpha}$ | Density of phase $\alpha$ (o, w, g) | kg/m³ |
| $S_{\alpha}$ | Saturation of phase $\alpha$ (o, w, g) | dimensionless |
| $\sigma$ | Non-linear activation function | - |
| $t$ | Time | s |
| $\theta$ | Set of learnable parameters in a neural network/operator | - |
| $\mathbf{u}_{\alpha}$ | Darcy velocity **vector** of phase $\alpha$ | m/s |
| $u(x,t)$ | Output solution function from the neural operator | - |
| $\mathcal{U}(\Omega)$ | Banach space of output (solution) functions | - |
| $v_t(x)$ | Function representation at layer $t$ within a neural operator | - |
| $W$ | Local linear transformation (weight matrix) in an FNO layer | - |
| $x, y, z$ | Spatial coordinates | m |
| $\nabla$ | Gradient operator | 1/m |
| $\nabla \cdot$ | Divergence operator | 1/m |

---

## References
[1] O. Heaviside: "Operators in mathematical physics", Proc. Roy. Soc. A. 52 (1893), p. 504.  
[2] H.-J. Glaeske, A.P. Prudnikov, K.A. Skòrnik: Operational Calculus and Related Topics, Taylor & Francis, 2006.  
[3] Li, Zongyi, Nikola Kovachki, Kamyar Azizzadenesheli, Burigede Liu, Kaushik Bhattacharya, Andrew Stuart, and Anima Anandkumar. “Neural Operator: Graph Kernel Network for Partial Differential Equations.” arXiv, 2020. https://doi.org/10.48550/ARXIV.2003.03485.  
[4] Li, Zongyi, Nikola Kovachki, Kamyar Azizzadenesheli, Burigede Liu, Kaushik Bhattacharya, Andrew Stuart, and Anima Anandkumar. “Fourier Neural Operator for Parametric Partial Differential Equations.” arXiv, 2020. https://doi.org/10.48550/ARXIV.2010.08895.